# Evaluation of LLM Agents ( LangChain )

Agents are often useful in the RAG setting to retrieve real-time information to
be used for question answering.

This example utilizes the openai functions agent to reliably call and return
structured responses from particular tools. Certain OpenAI models have been
fine-tuned for this capability to detect when a particular function should be
called and respond with the inputs requred for that function. Compared to a
ReACT framework that generates reasoning and actions in an interleaving manner,
this strategy can often be more reliable and consistent.

In either case - as the questions change over time, different agents may be
needed to retrieve the most useful context. In this example you will create a
langchain agent and use TruLens to identify gaps in tool coverage. By quickly
identifying this gap, we can quickly add the missing tools to the application
and improve the quality of the answers.

[![Open In
Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/frameworks/langchain/langchain_agents.ipynb)

### Import from LangChain and TruLens

### Install additional packages

In addition to trulens-eval and langchain, we will also need additional packages: `yfinance` and `google-search-results`.

In [ ]:
! pip install \
    "boto3" \
    "litellm" \
    "trulens_eval==0.20.2" \
    "langchain>=0.0.248" \
    "openai>=1.0" \
    "yfinance>=0.2.27" \
    "google-search-results>=2.4.2" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.2 MB/s eta 0:00:00


In [38]:
from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval import TruChain
from trulens_eval.feedback import OpenAI as fOpenAI

tru = Tru()

from datetime import datetime
from datetime import timedelta
from typing import Type

from langchain import SerpAPIWrapper
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool

import openai
from pydantic import BaseModel
from pydantic import Field
import yfinance as yf

## Setup
### Add API keys
For this quickstart you will need Open AI and SERP API keys.

In [39]:
import os
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
os.environ["SERPAPI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

### Create agent with search tool

In [40]:
search = SerpAPIWrapper()
search_tool = Tool(
    name="Search",
    func=search.run,
    description="useful for when you need to answer questions about current events"
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

tools = [search_tool]

agent = initialize_agent(
    tools, llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

### Set up Evaluation

In [42]:
class OpenAI_custom(fOpenAI):
    def no_answer_feedback(self, question: str, response: str) -> float:
        return float(self.endpoint.client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "Does the RESPONSE provide an answer to the QUESTION? Rate on a scale of 1 to 10. Respond with the number only."},
            {"role": "user", "content": f"QUESTION: {question}; RESPONSE: {response}"}
        ]
    ).choices[0].message.content) / 10

custom = OpenAI_custom()

# No answer feedback (custom)
f_no_answer = Feedback(custom.no_answer_feedback).on_input_output()

✅ In no_answer_feedback, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In no_answer_feedback, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [44]:
tru_agent = TruChain(
    agent,
    app_id="Search_Agent",
    feedbacks = [f_no_answer]
)

In [45]:
prompts = [
    "What company acquired MosaicML?",
    "What's the best way to travel from NYC to LA?",
    "How did the change in the exchange rate during 2021 affect the stock price of US based companies?",
    "Compare the stock performance of Google and Microsoft",
    "What is the highest market cap airline that flies from Los Angeles to New York City?",
    "I'm interested in buying a new smartphone from the producer with the highest stock price. Which company produces the smartphone I should by and what is their current stock price?"
]

with tru_agent as recording:
    for prompt in prompts:
        agent(prompt)



> Entering new AgentExecutor chain...

Invoking: `Search` with `company that acquired MosaicML`


DatabricksDatabricks acquired MosaicML.

> Finished chain.


> Entering new AgentExecutor chain...
The best way to travel from New York City (NYC) to Los Angeles (LA) depends on your preferences and budget. Here are some common options:

1. **Flight**: The fastest way to travel between NYC and LA is by taking a direct flight. Several airlines offer non-stop flights between the two cities, which can take around 5-6 hours.

2. **Train**: If you prefer a more scenic route, you can take an Amtrak train from NYC to LA. The journey takes around 2-3 days and offers beautiful views of the countryside.

3. **Bus**: Taking a bus is a more budget-friendly option, but it also takes longer than flying or taking the train. Several bus companies offer routes between NYC and LA.

4. **Driving**: If you enjoy road trips, you can drive from NYC to LA. The journey takes around 40-45 hours, depending on the

After running the first set of prompts, we notice that our agent is struggling with questions around stock performance.

In response, we can create some custom tools that use yahoo finance to get stock performance information.

### Define custom functions

In [46]:
def get_current_stock_price(ticker):
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="1d")
    return {"price": recent.iloc[0]["Close"], "currency": ticker_data.info["currency"]}


def get_stock_performance(ticker, days):
    """Method to get stock price change in percentage"""

    past_date = datetime.today() - timedelta(days=days)
    ticker_data = yf.Ticker(ticker)
    history = ticker_data.history(start=past_date)
    old_price = history.iloc[0]["Close"]
    current_price = history.iloc[-1]["Close"]
    return {"percent_change": ((current_price - old_price) / old_price) * 100}

### Make custom tools

In [47]:
class CurrentStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price"""

    ticker: str = Field(description="Ticker symbol of the stock")


class CurrentStockPriceTool(BaseTool):
    name = "get_current_stock_price"
    description = """
        Useful when you want to get current stock price.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = CurrentStockPriceInput

    def _run(self, ticker: str):
        price_response = get_current_stock_price(ticker)
        return price_response

current_stock_price_tool = CurrentStockPriceTool()

class StockPercentChangeInput(BaseModel):
    """Inputs for get_stock_performance"""

    ticker: str = Field(description="Ticker symbol of the stock")
    days: int = Field(description="Timedelta days to get past date from current date")


class StockPerformanceTool(BaseTool):
    name = "get_stock_performance"
    description = """
        Useful when you want to check performance of the stock.
        You should enter the stock ticker symbol recognized by the yahoo finance.
        You should enter days as number of days from today from which performance needs to be check.
        output will be the change in the stock price represented as a percentage.
        """
    args_schema: Type[BaseModel] = StockPercentChangeInput

    def _run(self, ticker: str, days: int):
        response = get_stock_performance(ticker, days)
        return response

stock_performance_tool = StockPerformanceTool()

### Give our agent the new finance tools

In [48]:
tools = [search_tool, current_stock_price_tool, stock_performance_tool]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

### Set up Tracking + Eval

In [49]:
tru_agent = TruChain(agent, app_id = "Search_Agent_v2", feedbacks = [f_no_answer])

## Test the new agent

In [50]:
# wrapped agent can act as context manager
with tru_agent as recording:
    for prompt in prompts:
        agent(prompt)



> Entering new AgentExecutor chain...

Invoking: `Search` with `company that acquired MosaicML`


DatabricksDatabricks acquired MosaicML.

> Finished chain.


> Entering new AgentExecutor chain...
The best way to travel from New York City (NYC) to Los Angeles (LA) depends on your preferences for cost, time, and comfort. Here are some common ways to travel between the two cities:

1. **By Air**: The fastest way to travel from NYC to LA is by taking a direct flight. There are multiple airlines that operate flights between John F. Kennedy International Airport (JFK) in NYC and Los Angeles International Airport (LAX) in LA. The flight duration is around 5 to 6 hours.

2. **By Train**: You can also consider taking an Amtrak train from NYC to LA. The Amtrak's "Southwest Chief" route runs between New York Penn Station and Union Station in Los Angeles. This option is more scenic but takes longer than flying.

3. **By Bus**: If you're looking for a budget-friendly option, you can take a long-

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
npx: installed 22 in 4.126s

Go to this url and submit the ip given here. your url is: https://pink-pans-live.loca.lt

  Submit this IP Address: 35.237.211.80



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.